# Test Envoiriment for the  functions in the module

[How To Use Conda Environment In a Jupyter Notebook
](https://saturncloud.io/blog/how-to-use-conda-environment-in-a-jupyter-notebook/#:~:text=Using%20a%20Conda%20Environment%20in%20Jupyter%20Notebook&text=This%20package%20allows%20you%20to,be%20used%20in%20Jupyter%20Notebook.&text=This%20will%20create%20a%20new%20kernel%20for%20the%20“myenv”%20environment,be%20used%20in%20Jupyter%20Notebook.)

## Train

In [ ]:
#!python main.py --model --datasetpath --state "train" --pretrained_path --output_path

## Test

In [8]:
from loss import *
from Metrics import *
from torch.nn.functional import mse_loss
from kornia.losses import ssim_loss
from numpy import mean 
from time import time
import lpips

class ssim_1(nn.Module):
    def __init__(self, window_size):
        super(ssim_1,self).__init__()
        self.window_size = window_size
    def forward(self, input, target):
        return ssim_loss(input, target, self.window_size)

class mse_1(nn.Module):
    def __init__(self):
        super(mse_1,self).__init__()

    def forward(self, input, target):
        return mse_loss(input, target)

gt_image = torch.randn(1, 3, 64, 64, requires_grad=True)
t_image = torch.randn(1, 3, 64, 64, requires_grad=True)
mean_val = mean(t_image.detach().numpy())

# Example usage of the Dark Channel Prior loss function
#find patch size
patch_size = 5
print(mean)
losses = {
    "DarkChannelPriorLoss": DarkChannelPriorLoss(patch_size),
    "L_exp": L_exp(patch_size,mean_val),
    "L_color": L_color(),
    "mse_loss":mse_1(),
    "light_loss": light_loss2(),
    "color_loss": color_loss2(),
    "L1Loss": L1Loss(),
    "SSIM loss":ssim_1(window_size=11),
    "lch_loss": lch_Loss(),
    "lab_loss": lab_Loss(),
    "loss_fn_alex" : lpips.LPIPS(net='alex'), # best forward scores
    "loss_fn_vgg" : lpips.LPIPS(net='vgg'), # closer to "traditional" perceptual loss, when used for optimization
    "loss_squeeze" : lpips.LPIPS(net='squeeze') # lightest, faster, narrower, lower quality
}

for name, loss in losses.items():
    start_time = time()
    if name== "loss_fn_vgg" or name == "loss_squeeze" or name == "loss_fn_alex" or name == "L1Loss" or name == "light_loss" or name == "color_loss" or name == "SSIM loss" or name == "color_loss" or name == "mse_loss" or name == "lab_loss" or name == "lch_loss":
        loss = loss(gt_image, t_image)
    else:
        loss = loss(t_image)
    end_time = time()
    print(name, ":", loss.item(), "// Time:", end_time-start_time)


<function mean at 0x77d6e03c6020>
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/lpips/weights/v0.1/vgg.pth
Setting up [LPIPS] perceptual loss: trunk [squeeze], v[0.1], spatial [off]
Loading model from: /home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/lpips/weights/v0.1/squeeze.pth
DarkChannelPriorLoss : -4.091731548309326 // Time: 0.000232696533203125
L_exp : 0.011746272444725037 // Time: 0.0001773834228515625
L_color : 3.982763519161381e-05 // Time: 0.00012874603271484375
mse_loss : 1.9858440160751343 // Time: 6.437301635742188e-05
light_loss : 0.7579351663589478 // Time: 0.00023174285888671875
color_loss : 0.9887773990631104 // Time: 0.002427339553833008
L1Loss : 1.1202781200408936 // Time: 4.649162292480469e-05
SSIM loss : 0.4979829788208008 // Time: 0.003293275833129883
lch_loss : 154.82911682128906 // Time: 0.011107921600341797
lab_loss : 25146.662109375 // Time: 0.0026063919067382812
loss_fn_alex : 0.11614850163459778 // Time: 0.007351398468017578
loss_fn

/home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import torch
import torch.nn as nn

class L1Loss(nn.Module):
    def __init__(self):
        super(L1Loss, self).__init__()

    def forward(self, input, target):
        # Calcula a diferença absoluta entre o input e o target
        abs_diff = torch.abs(input - target)
        # Calcula a média da diferença absoluta
        loss = torch.mean(abs_diff)
        return loss

# Exemplo de uso da função de perda L1
loss_function = L1Loss()

# Tensors de exemplo
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5, requires_grad=False)

# Calcula a perda
loss = loss_function(input, target)

print("L1 Loss:", loss.item())

L1 Loss: 1.2468688488006592


In [1]:
from src.split_data import split_dataset

dataset_size = 1000  # Tamanho do dataset
proportions = (0.7, 0.2, 0.1)  # Proporções para treino, teste e validação
dataset_path = "data/UDWdata/SUIM/train/images"  # Caminho para a pasta do dataset

result = split_dataset(dataset_size, proportions, dataset_path)

print(f"Imagens para treino: {result['train_size']}")
print(f"Imagens para teste: {result['test_size']}")
print(f"Imagens para validação: {result['val_size']}")

print("Endereços das imagens para treino:")
for img in result['train_images']:
    print(img)

print("Endereços das imagens para teste:")
for img in result['test_images']:
    print(img)

print("Endereços das imagens para validação:")
for img in result['val_images']:
    print(img)

1525
1000


TypeError: 'NoneType' object is not subscriptable

In [68]:
import os
import glob

def load_image_paths(dataset_path, dataset="all",task="train"):
    """
    dataset_path: endereço do dataset raiz
    dataset: "all", "UIEB", "RUIE", "SUIM"
    task: "train", "val"

    """
    image_paths = []
    if dataset == "all":
        # Constrói os padrões de caminho para os arquivos .jpg e .png dentro das pastas train e train/images
        pattern1_jpg = os.path.join(dataset_path, "*", f"{task}", "*.jpg")
        pattern2_jpg = os.path.join(dataset_path, "*", f"{task}", "images", "*.jpg")
        pattern1_png = os.path.join(dataset_path, "*", f"{task}", "*.png")
        pattern2_png = os.path.join(dataset_path, "*", f"{task}", "images", "*.png")
        pattern3_jpg = os.path.join(dataset_path, "*", "*",f"{task}", "*.jpg")

        
        # Encontra todos os arquivos .jpg e .png correspondentes aos padrões
        image_paths.extend(glob.glob(pattern1_jpg))
        image_paths.extend(glob.glob(pattern2_jpg))
        image_paths.extend(glob.glob(pattern1_png))
        image_paths.extend(glob.glob(pattern2_png))
        image_paths.extend(glob.glob(pattern3_jpg))
    elif dataset == "SUIM":
         pattern2_jpg = os.path.join(dataset_path, "*", f"{task}", "images", "*.jpg")
         image_paths.extend(glob.glob(pattern2_jpg))
    elif dataset == "UIEB":
        pattern1_png = os.path.join(dataset_path, "*", f"{task}", "*.png")
        image_paths.extend(glob.glob(pattern1_png))
    elif dataset == "RUIEB":
        pattern3_jpg = os.path.join(dataset_path, "*", "*",f"{task}", "*.jpg")
        image_paths.extend(glob.glob(pattern3_jpg))
        
    return image_paths
#dataset_path = "data/UDWdata/"  # Caminho para a pasta do dataset

#data = load_image_paths(dataset_path,dataset="all")

#print(len(data),data)

6130 ['data/UDWdata/SUIM/train/images/d_r_301_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1098_.jpg', 'data/UDWdata/SUIM/train/images/f_r_2060_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1134_.jpg', 'data/UDWdata/SUIM/train/images/w_r_124_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1073_.jpg', 'data/UDWdata/SUIM/train/images/w_r_1_.jpg', 'data/UDWdata/SUIM/train/images/f_r_676_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1424_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1063_.jpg', 'data/UDWdata/SUIM/train/images/f_r_759_.jpg', 'data/UDWdata/SUIM/train/images/w_r_49_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1609_.jpg', 'data/UDWdata/SUIM/train/images/d_r_270_.jpg', 'data/UDWdata/SUIM/train/images/w_r_143_.jpg', 'data/UDWdata/SUIM/train/images/w_r_127_.jpg', 'data/UDWdata/SUIM/train/images/w_r_133_.jpg', 'data/UDWdata/SUIM/train/images/f_r_877_.jpg', 'data/UDWdata/SUIM/train/images/n_l_89_.jpg', 'data/UDWdata/SUIM/train/images/f_r_2075_.jpg', 'data/UDWdata/SUIM/train/images/f_r_1088_.jpg', 'd

## Inference